https://huggingface.co/docs/transformers/tasks/translation

In [1]:
%pip install transformers datasets evaluate sacrebleu

     -------------------------------------- 468.7/468.7 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 81.4/81.4 kB 2.3 MB/s eta 0:00:00
     -------------------------------------- 118.9/118.9 kB 2.3 MB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 145.4/145.4 kB 2.2 MB/s eta 0:00:00
     -------------------------------------- 134.3/134.3 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 20.6/20.6 MB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 2.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from datasets import load_dataset

books = load_dataset('json', data_files='../test_dataset/eng_tok.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/Andrey Chirkin/.cache/huggingface/datasets/json/default-0af0fa0effee8e2c/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\Andrey Chirkin\.cache\huggingface\token
Login successful


In [4]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

c:\Users\Andrey Chirkin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [19]:
books = books["train"].train_test_split(test_size=0.2)

In [8]:
books["train"][0]

{'id': '7223',
 'translation': {'en': '"La! You are so strange!',
  'fr': '– Mon Dieu ! que vous etes étrange !'}}

In [22]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    return model_inputs

In [23]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/81334 [00:00<?, ? examples/s]

Map:   0%|          | 0/20334 [00:00<?, ? examples/s]

In [32]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [34]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [33]:
import evaluate

metric = evaluate.load("sacrebleu")

NameError: name '_C' is not defined

In [35]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

RuntimeError: Failed to import transformers.training_args_seq2seq because of the following error (look up to see its traceback):
name '_C' is not defined

Mine

In [12]:
source_lang = "en"
target_lang = "tok"
prefix = "translate English to Toki-pona: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [10]:
import json

In [ ]:
with open("..\corpora\eng_tok.json", 'r', encoding="utf-8") as f:
    sentences = json.loads(f.read())
    sentences = list(sentences["train"])

tokenized_sentences = list(map(preprocess_function, sentences))

In [14]:
tokenized_sentences